# Imports

In [1]:
import os
import sys
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple
import joblib

import pandas as pd
from tqdm.notebook import tqdm

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "_0_Constants_and_Utils"))


from category_utils import convert_to_list, get_batches, normalise_text
from database_utils import (
    connect_to_database,
    execute_queries,
    form_connection_params,
    get_dataframe_from_query,
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chekm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chekm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chekm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Constants

In [2]:
QUERY_TEXT = """
SELECT 
    c.conversation_id, 
    CONCAT(t1.full_text, ' ', t2.full_text) AS combined_full_text
FROM 
    Conversations AS c
JOIN 
    Tweets AS t1
ON 
    c.tweet_id = t1.tweet_id
JOIN 
    ConversationsCategory AS cc
ON 
    c.conversation_id = cc.conversation_id
JOIN 
    Conversations AS c2
ON 
    c.conversation_id = c2.conversation_id 
AND 
    c2.tweet_order = 2
JOIN 
    Tweets AS t2
ON 
    c2.tweet_id = t2.tweet_id
WHERE 
    c.tweet_order = 1
"""

# Loading

In [3]:
# Set local = False if you want to query the online MySQL database
local = True
connection_params = form_connection_params(local, True)

In [4]:
test_data = get_dataframe_from_query(QUERY_TEXT, connection_params, local, index_col="conversation_id")
test_data

,combined_full_text
conversation_id,
1,@nealrach @VirginAtlantic Siiiigh.... Still no...
2,We’re waiving change fees for customers who ha...
3,@katiewithani Please be assured if your flight...
4,@Grenzmauer75 @elliotday @easyJet Exactly. Do ...
5,@RosamariaP3 Hola Rosa ✌ Siento que aún no hay...
...,...
458720,@airfrance j'ai mis une bombe dans un a avion...
458721,@Ryanair What if I make it into a Turban then?...
458722,@AmericanAir Please help me!! I've fallen on ...


In [5]:
test_data['cleaned_text'] = test_data['combined_full_text'].apply(normalise_text)

In [6]:
test_data

,combined_full_text,cleaned_text
conversation_id,,
1,@nealrach @VirginAtlantic Siiiigh.... Still no...,siiiigh still idea theyre back officially...
2,We’re waiving change fees for customers who ha...,’ waiving change fee customer travel plan ...
3,@katiewithani Please be assured if your flight...,please assured flight cancelled contact...
4,@Grenzmauer75 @elliotday @easyJet Exactly. Do ...,exactly plan go bankrupt pay shareholder ...
5,@RosamariaP3 Hola Rosa ✌ Siento que aún no hay...,hola rosa ✌ siento que aún hayas recibido el ...
...,...,...
458720,@airfrance j'ai mis une bombe dans un a avion...,jai mi une bombe dans un avion n prenons ce m...
458721,@Ryanair What if I make it into a Turban then?...,make turban sorry harry luggage bi...
458722,@AmericanAir Please help me!! I've fallen on ...,please help ive fallen one plain please re...


# Categorization

### Prepare the data for training and testing

In [9]:
SVM_MODEL = joblib.load("svm_model.joblib")
VECTORIZER = joblib.load("tfidf_vectorizer.joblib")
ENCODER = joblib.load("label_encoder.joblib")

def predict_categories(batch):
    tweet_vectors = VECTORIZER.transform(batch).toarray()
    predicted_labels = SVM_MODEL.predict(tweet_vectors)
    categories = ENCODER.inverse_transform(predicted_labels)
    return categories.tolist()

def get_category(df, text_column, batch_size=128, max_workers=4):
    texts = df[text_column].tolist()
    labels = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(predict_categories, texts[i:i + batch_size]) for i in range(0, len(texts), batch_size)]
        for future in futures:
            batch_labels = future.result()
            labels.extend(batch_labels)

    df["category"] = labels
    return df

In [10]:
df = pd.read_excel('clean_labels.xlsx').query("Category != 'Undefined category'")
test = df.copy()
test['our_guess'] = predict_categories(test['text'])
accuracy = (test['Category'] == test['our_guess']).mean()*100


print(f"Accuracy after fine-tuning: {accuracy:.2f}%")
test

Accuracy after fine-tuning: 98.84%


,text,Category,our_guess
0,al hilo de la demostración de este fenómeno v...,In-Flight Experience,In-Flight Experience
1,hi rachel price isnt available possible m...,Booking,Booking
2,thank showing around airbus first class s...,In-Flight Experience,In-Flight Experience
3,ejuqv oeizc easyjet europe landed lowi plan...,Flight Information Requests,Flight Information Requests
4,ezydn gezwd easyjet landed lowi planespotti...,Flight Information Requests,Flight Information Requests
...,...,...,...
2602,frequent flyer im extremely annoyed looking...,Frequent Flyer,Frequent Flyer
2603,definitely come top worst airline experienc...,Frequent Flyer,Frequent Flyer
2604,current flight information updated regularly...,Frequent Flyer,Frequent Flyer
2605,often see get lot complaint deal freque...,Frequent Flyer,Frequent Flyer


In [11]:
def get_category(df, text_column, batch_size=128, max_workers=4):
    texts = df[text_column].tolist()
    labels = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(predict_categories, texts[i:i + batch_size]) for i in range(0, len(texts), batch_size)]
        for future in futures:
            batch_labels = future.result()
            labels.extend(batch_labels)

    df["category"] = labels
    return df


def update_categories(
    batch: List[Tuple[str, str]], connection_params: dict, local: bool
) -> None:
    """
    Update full_text values for a batch of data in the local SQLite database.

    Args:
        batch: List of (full_text, tweet_id) pairs.
        db_path: The path to the SQLite database file.
    """
    update_query = "UPDATE ConversationsCategory SET category = ? WHERE conversation_id = ?"
    if not local:
        update_query = update_query.replace("?", "%s")
    with connect_to_database(connection_params, local) as connection:
        execute_queries(connection, [(update_query, batch)])

In [12]:
data_batches = get_batches(test_data[["cleaned_text"]], 10_000)

In [13]:
for batch in tqdm(data_batches, desc="Updating categories: "):
    df_categories = get_category(batch, "cleaned_text", 512, 10)
    update_categories(convert_to_list(df_categories), connection_params, local)

Updating categories:   0%|          | 0/46 [00:00<?, ?it/s]